In [1]:
import torch
import torch.nn as nn
import numpy as np

import datasets.datasets as dtset
import utils.losses as losses

from utils.weakener import Weakener
from models.model import MLP

from utils.trainig_testing import train_and_evaluate

In [2]:
Data = dtset.Torch_Dataset('mnist', batch_size = 16)
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll',pll_p=0.2)

In [3]:
train_X,train_y,test_X,test_y =  Data.get_data()
Weak.generate_weak(train_y) #z and w


(tensor([530, 511,  97,  ..., 403, 519,  69], dtype=torch.int32),
 tensor([[1., 0., 0.,  ..., 0., 1., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         ...,
         [0., 1., 1.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.]], dtype=torch.float64))

In [4]:
'''
import pickle
Dataset = [Data,Weak]
f = open("Experimental_results/Datasets.pkl","wb")
pickle.dump(Dataset,f)
f.close()
'''

'\nimport pickle\nDataset = [Data,Weak]\nf = open("Experimental_results/Datasets.pkl","wb")\npickle.dump(Dataset,f)\nf.close()\n'

In [5]:
Weak.z.shape

torch.Size([60000])

In [6]:
#Depending the model we are about to feed we will need either weak labels or virtual labels. 

In [7]:
Weak.virtual_labels(p=None, optimize = False, convex = True) #This is to create Virtual Labels


C:\Users\danibacaicoa\AppData\Roaming\Python\Python311\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


In [14]:
Weak.w

tensor([[1., 0., 0.,  ..., 0., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 1.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 0.]], dtype=torch.float64)

In [15]:
Data.include_weak(Weak.w)
#Data.include_virtual(Weak.v)

In [16]:
trainloader,testloader = Data.get_dataloader(weak_labels='weak')

In [17]:
examples = next(iter(trainloader))
for k in examples:
    print(k)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
        [1., 1., 0., 0., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0., 0

In [18]:
#Weak.V_matrix(Data.num_classes)

In [19]:

loss = losses.OSLCELoss()
overall_results = {}
overall_models = {}
epochs = 5
for i in range(2):
    mlp = MLP(Data.num_features,[Data.num_features],Data.num_classes, dropout_p=0.5, bn=True, activation =  'relu')
    optim = torch.optim.Adam(mlp.parameters(),lr=1e-2)
    mlp, results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss,num_epochs=epochs,sound=1)
    overall_results[i] = results
    overall_models[i] = mlp  

Epoch 1/5: Train Loss: 0.4216, Train Acc: 0.8566, Test Acc: 0.9395


KeyboardInterrupt: 

In [ ]:
'''
import pickle
Results_mnist_ce = [overall_results,overall_models]
f = open("Experimental_results/EM.pkl","wb")
pickle.dump(Results_mnist_ce,f)
f.close()
'''

In [ ]:
import matplotlib.pyplot as plt

tr_loss = np.array(overall_results[0]['train_loss'])
tr_acc = np.array(overall_results[0]['train_acc'])
te_acc = np.array(overall_results[0]['test_acc'])
for i in range(1):
    tr_loss = np.vstack((tr_loss,overall_results[i+1]['train_loss']))
    tr_acc = np.vstack((tr_acc,overall_results[i+1]['train_acc']))
    te_acc = np.vstack((te_acc,overall_results[i+1]['test_acc']))
plt.plot(np.mean(tr_loss,0))
plt.plot(np.mean(tr_loss,0)+np.std(tr_loss,0),'g--')
plt.plot(np.mean(tr_loss,0)-np.std(tr_loss,0),'g--')

In [ ]:
plt.plot(np.mean(te_acc,0))
plt.plot(np.mean(te_acc,0)+np.std(te_acc,0),'g--')
plt.plot(np.mean(te_acc,0)-np.std(te_acc,0),'g--')

In [ ]:
plt.plot(np.mean(tr_acc,0))
plt.plot(np.mean(tr_acc,0)+np.std(tr_acc,0),'g--')
plt.plot(np.mean(tr_acc,0)-np.std(tr_acc,0),'g--')